In [1]:
import hail as hl

In [2]:
import pandas as pd
import h5py
import h5py_cache
import sys 
import numpy as np
sys.path.insert(0, '../code/')
from importlib import reload 
import my_hail_helper as myhelper

In [3]:
myhelper = reload(myhelper)

In [4]:
covar, trait = myhelper.read_and_split_phenotype_csv(
    csv_path = '/vol/bmd/yanyul/GitHub/ptrs-ukb/output/query_phenotypes_cleaned_up.csv',
    pheno_names = 'ht,meaning',
    covar_names = 'pc1',
    indiv_id = 'eid',
    int_names = None,
    str_names = 'meaning,eid'
)

In [5]:
indiv_list = myhelper.read_indiv_list('/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/British-test-1.txt')

In [6]:
trait_sub = myhelper.subset_by_col(trait, 'eid', indiv_list)

In [7]:
indiv_in_sub_table = np.array(trait_sub['eid'].to_list()).astype('S10')

In [8]:
HDF5_CACHE = int(60 * (1024 ** 2))
f = h5py_cache.File(
    '/vol/bmd/yanyul/UKB/predicted_expression/predicted_expression.ukb_imp_x_ctimp_Whole_Blood.h5', 
    'r', 
    chunk_cache_mem_size = HDF5_CACHE, 
    w0 = 1.0, 
    dtype = 'float32'
)

In [9]:
f.keys()

<KeysViewHDF5 ['genes', 'pred_expr', 'samples']>

In [9]:
# pred_mat = f['pred_expr']

In [10]:
subset_idx_for_indiv_in_sub_table = np.where(np.isin(f['samples'][:], indiv_in_sub_table))

In [11]:
subset_idx_for_indiv_in_sub_table[0].shape

(5000,)

In [18]:
# f['pred_expr'][1:100].shape
np.array([i for i in range(10)])[::5]

array([0, 5])

In [ ]:
pred_mat_sub = f['pred_expr'][:, subset_idx_for_indiv_in_sub_table[0]]

In [8]:
test = hl.Table.from_pandas(pd.DataFrame(pred_mat[1:10,1:10]))

In [9]:
test.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    '0': float64 
    '1': float64 
    '2': float64 
    '3': float64 
    '4': float64 
    '5': float64 
    '6': float64 
    '7': float64 
    '8': float64 
----------------------------------------
Key: []
----------------------------------------
